In [62]:
from cdag import Graph, load_data, read_table
from personal import database_connection
import re, pandas as pd

In [63]:
# read HiX tables
# method 1: from CSV files
# method 2: from database
# connection = database_connection('hixacceptatie')
# iv = pd.read_sql(sql=f"select * from [hix_acc].[dbo].[config_instvars];", con=connection
iv = pd.read_csv('config_instvars.csv')
iv['VALUE'] = iv['VALUE'].fillna('X""')
# iv.groupby('INSTTYPE').count()
# C: 5671, D: 351, G: 6382, L: 70606, U: 904852
# type L has only a few possibilities for VALUE: LT©, LTTTTTT, CT
# type U has more options for value (including XML strings)
# types D and G always have owner=CHIPSOFT

In [64]:
# read HiX tables (continued)
wc = pd.read_csv('config_workcontext.csv')
wc.set_index('Id', inplace=True)
ws = pd.read_csv('config_wcsegments.csv')
ws.set_index('ConfiguredWorkContextId', inplace=True)
wsa = ws.loc[ws['Disabled'] == False]
wcs = wc.join(wsa, how='inner')
gr = pd.read_csv('ziscon_groepen.csv')
#us = pd.read_csv('ziscon_user.csv',
#                 dtype={'FUNKTIE':'string', 'BEHEERDER':'boolean', 'WACHTWOORD':'string',
#                        'DISABLED': 'boolean', 'UZINR': 'string', 'ROLE': 'string', 'WRONGCHECKS': 'string'})

In [65]:
# read reference table
tg = pd.read_csv('chipsoft_logic_table_guids.csv', sep=';')
table_name = {}
for elt in tg.to_dict('records'):
    table_name[elt['classguid']] = elt['classid']

In [82]:
guid_rx = re.compile(r'\{[A-Z0-9-]+?\}')
def resolve_guid(match):
    return table_name.get(match.group(0), '<unknown>')

In [80]:
# Unicode dingbats, open and closed circled digits
OD1, CD1 = '\u2780', '\u2776'
OD2, CD2 = '\u2781', '\u2777'
OD3, CD3 = '\u2782', '\u2778'
level1_rx = re.compile(r'"([\u2782\u2778\u2781\u2777\x20\x21\x23-\x7E]+?)"')
level2_rx = re.compile(r'""([\u2782\u2778\x20\x21\x23-\x7E]+?)""')
level3_rx = re.compile(r'""""([\x20\x21\x23-\x7E]+?)""""')

def tokenize(s): 
    if s.startswith('C'):
        s3 = level3_rx.sub('\u2782\g<1>\u2778', s.lstrip('C').rstrip('\x01\u00a9'))
        s2 = level2_rx.sub('\u2781\g<1>\u2777', s3)
        s1 = level1_rx.sub('\u2780\g<1>\u2776', s2)
        return s1
    else:
        return s
    
def parse(s):
    t = tokenize(guid_rx.sub(resolve_guid, s))
    if t.startswith('\u2780') and t.endswith('\u2776'):
        # TODO: handle levels 2 and 3
        return t[1:-1].split('\u2776,\u2780')
    else:
        print(f"{t}: begin={t[0]} end={t[-1]}")
        return []

In [101]:
# x = iv.loc[(iv['OWNER'] == '@GBEH') & iv['NAAM'].str.contains('DOSS')].to_dict('records')
# x[0]
# groen        = ja
# rood         = nee, tenzij via overerving
# geel         = nee, tenzij met rechtgebonden werkcontext | met zwart kader: werkcontext is ingesteld
# blauw        = nee, tenzij met standaard werkcontext | met zwart kader: werkcontext is ingesteld
# donkerkgrijs = nooit
# BEH: doss_noodprexemp geel value=CM
#      doss_noodprsee   groen value=CT
#      ond_rechten/widocarc CSSSSSF 5xblauw 1xrood
# kijk ook bij Document. inverted=true -> alles behalve
iv.loc[(iv['OWNER'] == '@GBEH') & iv['NAAM'].str.contains('WI_EPD')]# & iv['SPECCODE'].str.contains('WI')]

In [86]:
parse(x[0]['VALUE'])

In [104]:
wcs.loc[(wcs['OwnerId']=='BEH') & wcs['SegmentClassId'].str.contains('WI')]

In [70]:
#occurring, missing = {}, set()
#sep = '-' * 10
#for k, elt in enumerate(type_c.to_dict('records')):
#    value = guid_rx.sub(resolve_guid, elt['VALUE'])
#    # matches = guid_rx.findall(value)
#    if k < 5:
#        print(f"{sep}\nvalue {k}")
#        tokens = tokenize(value)
#        obj = parse(tokens)
#        for elt in obj:
#            print('\t'+elt)